In [1]:
# Import dependencies
import pandas as pd

# Ignore Python warnings about past model versions
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import dataset and display first rows of dataset
df = pd.read_csv("data/middle_school_features.csv")
display(df)

,dbn,school_name,applications,acceptances,per_pupil_spending,avg_class_size,asian_percent,black_percent,hispanic_percent,multiple_percent,...,effective_school_leadership,strong_family_community_ties,trust,disability_percent,poverty_percent,ESL_percent,school_size,student_achievement,reading_scores_exceed,math_scores_exceed
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,6,0,24890.0,20.15,3.3,31.8,61.3,0.9,...,2.45,3.33,2.32,36.6,97.6,7.5,333.0,2.70,0.33,0.26
1,01M140,P.S. 140 NATHAN STRAUS,6,0,23536.0,24.56,5.7,12.4,78.7,0.5,...,4.11,3.12,3.26,31.0,87.3,9.7,371.0,2.71,0.31,0.24
2,01M184,P.S. 184M SHUANG WEN,67,23,16206.0,29.69,69.4,3.7,13.5,6.1,...,2.75,3.38,2.91,16.5,67.5,15.1,684.0,4.28,0.68,0.74
3,01M188,P.S. 188 THE ISLAND SCHOOL,0,0,21960.0,24.09,1.8,29.3,66.1,0.0,...,4.99,4.99,4.99,29.1,91.5,15.9,433.0,4.54,0.28,0.24
4,01M301,"TECHNOLOGY, ARTS, AND SCIENCES STUDIO",11,0,25444.0,15.80,3.8,32.1,54.7,3.8,...,NaN,NaN,NaN,42.5,82.1,2.8,106.0,NaN,0.33,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,84X538,ICAHN CHARTER SCHOOL 5,20,1,NaN,NaN,4.1,58.0,31.0,1.3,...,3.38,3.38,3.51,6.9,65.2,0.9,319.0,3.31,0.33,0.33
590,84X703,BRONX PREPARATORY CHARTER SCHOOL,22,1,NaN,NaN,0.3,47.7,50.5,0.8,...,3.56,3.75,3.66,12.2,82.9,7.8,715.0,4.33,0.55,0.56
591,84X704,KIPP ACADEMY CHARTER SCHOOL,23,1,NaN,NaN,0.2,41.3,56.5,1.4,...,3.01,2.15,3.06,17.5,87.4,10.8,1074.0,2.98,0.60,0.61
592,84X706,HARRIET TUBMAN CHARTER SCHOOL,24,1,NaN,NaN,0.4,62.9,34.6,1.3,...,NaN,NaN,NaN,8.8,79.1,11.2,680.0,2.87,0.69,0.73


# Data Exploration and Cleaning 

In [3]:
# Drop school ID code which is an unnecessary feature
df = df.drop("dbn", axis = 1)

# Display columns that have missing values and display sum of missing values
missing = df.isnull().sum()
missing[missing > 0]

per_pupil_spending              121
avg_class_size                  121
asian_percent                     2
black_percent                     2
hispanic_percent                  2
multiple_percent                  2
white_percent                     2
rigorous_instruction             43
collaborative_teachers           43
supportive_environment           43
effective_school_leadership      43
strong_family_community_ties     45
trust                            45
disability_percent                2
poverty_percent                   2
ESL_percent                       2
school_size                       2
student_achievement              47
reading_scores_exceed             8
math_scores_exceed                8
dtype: int64

In this dataset, data for **per_pupil_spending** and for **avg_class_size** is missing for all charter schools. However given that the 109 charter schools represented in this dataset are all in the last rows of the dataset, I was able to split the dataset into two groups: public schools and charter schools. I then calculated the median value for public schools and then extrapolate this median value for all charter schools. Additionally there was missing data for those two rows for 12 public schools, which I also inputted the median values for both features for those schools with missing values. 

In [4]:
# Split the dataset into school category based on row number
public_schools = df[:-109]
charter_schools = df[-109:]

# Compute median values for both missing categories for public schools
med_pupil_spending_public = public_schools["per_pupil_spending"].median()
med_avg_class_size_public = public_schools["avg_class_size"].median()

# Impute missing values for charter schools using median public school values
charter_schools["per_pupil_spending"].fillna(med_pupil_spending_public, inplace = True)
charter_schools["avg_class_size"].fillna(med_avg_class_size_public, inplace = True)

In [5]:
# Combine the public and charter school data back together
cleaned_df = pd.concat([public_schools, charter_schools])

# Determine how many missing values there are now for these two features
cleaned_df.isnull().sum()[["per_pupil_spending", "avg_class_size"]]

per_pupil_spending    12
avg_class_size        12
dtype: int64

In [6]:
# Impute median values for public schools with missing values
cleaned_df["per_pupil_spending"].fillna(med_pupil_spending_public, inplace = True)
cleaned_df["avg_class_size"].fillna(med_avg_class_size_public, inplace = True)

# Once again determine if there are any more missing values for these two features
cleaned_df.isnull().sum()[["per_pupil_spending", "avg_class_size"]]

per_pupil_spending    0
avg_class_size        0
dtype: int64

In [7]:
# Display other features which contain missing values
missing_features = missing[missing > 0].index

# Impute missing values within these columns using median value
for feature in missing_features:
    median = cleaned_df[feature].median()
    cleaned_df[feature].fillna(median, inplace = True)

In [8]:
# Check to see if there are any more missing values within the dataset
missing = cleaned_df.isnull().sum()
missing

school_name                     0
applications                    0
acceptances                     0
per_pupil_spending              0
avg_class_size                  0
asian_percent                   0
black_percent                   0
hispanic_percent                0
multiple_percent                0
white_percent                   0
rigorous_instruction            0
collaborative_teachers          0
supportive_environment          0
effective_school_leadership     0
strong_family_community_ties    0
trust                           0
disability_percent              0
poverty_percent                 0
ESL_percent                     0
school_size                     0
student_achievement             0
reading_scores_exceed           0
math_scores_exceed              0
dtype: int64

In [9]:
# Export the cleaned dataset to be used for model training
path = "data/cleaned_school_data.csv"
cleaned_df.to_csv(path, index = False)